In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
import logging

logging.basicConfig(level=logging.INFO)

next_overview_generator = AcquisitionTaskGenerator('image',
                         LocationRemover(JSONSettingsLoader('config_json/test2color_overview.json')),
                         SpiralOffsetGenerator(move_size=[50e-6, 50e-6], start_position=get_current_stage_coords()))

pipeline = AcquisitionPipeline(data_save_path='acquisition_data/test', hierarchy_levels=['image'])

pipeline.add_callback(next_overview_generator, 'image')

pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(20))

pipeline.run(initial_callback=next_overview_generator)

In [16]:
from skimage.filters import threshold_otsu
from scipy.ndimage import gaussian_filter, label
from skimage.segmentation import clear_border
from skimage.morphology import dilation, disk


def segment(img):
    g = gaussian_filter(img.astype(float), 5)
    t = max(3, threshold_otsu(g))
    labels, _ = label(g > t)
    labels = clear_border(labels)
    labels = dilation(labels, disk(3))
    return labels


In [ ]:
from matplotlib import pyplot as plt

img = pipeline.data[(0,)].data[0][0].squeeze()
plt.imshow(segment(img), cmap='turbo', interpolation='nearest')

In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.callback_buildingblocks.parameter_filtering import LocationKeeper, LocationRemover
from autosted.callback_buildingblocks.data_selection import NewestSettingsSelector
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.detection.roi_detection import SegmentationWrapper


pipeline = AcquisitionPipeline(data_save_path='acquisition_data/test', hierarchy_levels=['overview', 'detail'])

next_overview_generator = AcquisitionTaskGenerator('overview',
                         JSONSettingsLoader('config_json/test2color_overview.json'),
                         SpiralOffsetGenerator(move_size=[50e-6, 50e-6], start_position=get_current_stage_coords()))

detail_generator = AcquisitionTaskGenerator('detail',
                                            LocationRemover(JSONSettingsLoader('config_json/test2color_detail.json')),
                                            LocationKeeper(NewestSettingsSelector()),
                                            SegmentationWrapper(segment, offset_parameters='scan')
                                            )


pipeline.add_callback(next_overview_generator, 'overview')
pipeline.add_callback(detail_generator, 'overview')

pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(20))

pipeline.run(initial_callback=next_overview_generator)

In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.taskgeneration.task_filtering import AlreadyImagedFOVFilter
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.callback_buildingblocks.parameter_filtering import LocationKeeper, LocationRemover
from autosted.callback_buildingblocks.data_selection import NewestSettingsSelector
from autosted.callback_buildingblocks.stitched_data_selection import StitchedNewestDataSelector
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.detection.roi_detection import SegmentationWrapper


pipeline = AcquisitionPipeline(data_save_path='acquisition_data/test', hierarchy_levels=['overview', 'detail'])

next_overview_generator = AcquisitionTaskGenerator('overview',
                         JSONSettingsLoader('config_json/test2color_overview.json'),
                         SpiralOffsetGenerator(move_size=[40e-6, 40e-6], start_position=get_current_stage_coords()))

detail_generator = AcquisitionTaskGenerator('detail',
                                            LocationRemover(JSONSettingsLoader('config_json/test2color_overview.json')),
                                            LocationKeeper(NewestSettingsSelector(pipeline, 'overview')),
                                            SegmentationWrapper(
                                                segment, 
                                                data_source_callback=StitchedNewestDataSelector(pipeline, 'overview', register_tiles=False, offset_parameters='scan'),
                                                offset_parameters='scan')
                                            )
detail_generator.add_task_filters(AlreadyImagedFOVFilter(pipeline, 'detail', 0.5, True))


pipeline.add_callback(next_overview_generator, 'overview')
pipeline.add_callback(detail_generator, 'overview')

pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(20))

pipeline.run(initial_callback=next_overview_generator)